In [1]:
import torch
import numpy as np

# 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

# 读取数据

In [3]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [4]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-1.1782, -0.2428],
        [-0.3110,  1.2945],
        [-0.2906,  0.0820],
        [ 1.1163, -0.8387],
        [-1.4377,  1.1562],
        [ 0.0547, -1.0209],
        [ 1.5980,  0.6503],
        [-0.1598, -0.2579],
        [ 0.3579, -0.4270],
        [-0.7177, -1.6190]]) tensor([ 2.6807, -0.8216,  3.3273,  9.2674, -2.6095,  7.7992,  5.1878,  4.7591,
         6.3727,  8.2692])


# 定义模型

In [5]:
from torch import nn


class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
        
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(nn.Linear(num_inputs, 1)
                    # 此处还可以传入其他层
                   )
                    
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module()

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([('linear', nn.Linear(num_inputs, 1))
                                 # ......
                                ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.3011, -0.6226]], requires_grad=True)
Parameter containing:
tensor([0.3748], requires_grad=True)


# 初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

# 定义损失函数

In [9]:
loss = nn.MSELoss()

# 定义优化函数

In [10]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [11]:
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)

AttributeError: 'Sequential' object has no attribute 'subnet1'

In [12]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

# 训练模型

In [13]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 12.070515
epoch 2, loss: 3.649066
epoch 3, loss: 0.821794


In [14]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.6932, -2.8783]], requires_grad=True)
4.2 Parameter containing:
tensor([3.5221], requires_grad=True)
